In [3]:
import glob
import re
import pandas as pd
from yattag import Doc
import unicodedata

In [4]:
doc, tag, text = Doc().tagtext()

In [5]:
txt_files = glob.glob('data/**/**/*.txt')
#txt_files = glob.glob('teste_manual.txt')

In [6]:
all_contracts = []
for txt in txt_files:
    instances = re.findall('aviso (?:de)? abertura (?:.*licitacao)([\s\S]*?)\.\n',
                            unicodedata.normalize('NFKD', open(txt, encoding='utf-8').read()), 
                            re.IGNORECASE)
    all_contracts.extend(instances)

In [10]:
regex_dict = {'processo': 'PROCESSO[\s\S]*(\d{5}[\.|\-]\d{8}/\d{4}-\d{2})', 
              'objeto': 'objeto[\s\S]*?:?([\s\S]*?)\.',
              'valor_estimado': 'valor[\s\S]*estimado?[\s\S]*?(\d+(?:\.\d+)*,\d\d)', 
              'prazo': 'prazo[s]?:?([\s\S]*)data', 
              'tipo': 'Tipo[:]?([\s\S]*?)\.',
              'data': 'data[\s\S]*?\.*(\d{2}/\d{2}/\d{4}[\s\S].*\d)|(\d+ de .*),',
}

df_dict = {
    'processo': [],
    'objeto': [],
    'valor_estimado': [],
    'prazo': [],
    'tipo': [],
    'data': [],
    'texto': [],
}

In [11]:
for contract in all_contracts:
    df_dict['texto'].append(contract)
    for field in regex_dict:
        match = re.search(regex_dict[field], contract, re.IGNORECASE)
        if match:
            res = tuple(x for x in match.groups() if x is not None)
            df_dict[field].append(res[0])
        else:
            df_dict[field].append('nan')

df = pd.DataFrame.from_dict(df_dict)
df.to_csv('abertura_regex.csv')